### Import Libraries

In [1]:
# Tensorflow
import tensorflow as tf
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras import layers, models
import tensorflow_hub as hub

import os
import warnings
warnings.filterwarnings("ignore")

### Rename Dataset

In [2]:
# def rename_images_in_directory(directory):
#     for subdir, _, files in os.walk(directory):
#         for i, file in enumerate(files):
#             file_ext = os.path.splitext(file)[1]
#             class_name = os.path.basename(subdir)
#             new_name = f"{class_name}_{i+1}{file_ext}"
#             old_file = os.path.join(subdir, file)
#             new_file = os.path.join(subdir, new_name)
#             os.rename(old_file, new_file)

# # Rename images in train and test directories
# train_dir = 'dataset/train'
# test_dir = 'dataset/test'

# # rename_images_in_directory(train_dir)
# rename_images_in_directory(test_dir)

### Load and Preprocess Data

In [11]:
# Define paths
train_dir = 'dataset/train'
test_dir = 'dataset/test'

# Load datasets
batch_size = 32
img_height = 224
img_width = 224

train_dataset = image_dataset_from_directory(train_dir,
                                             shuffle=True,
                                             batch_size=batch_size,
                                             image_size=(img_height, img_width))

test_dataset = image_dataset_from_directory(test_dir,
                                            shuffle=True,
                                            batch_size=batch_size,
                                            image_size=(img_height, img_width))

Found 793 files belonging to 20 classes.
Found 199 files belonging to 20 classes.


### Build, Train, and Evaluate the Model

In [12]:
# Create the base model from the pre-trained model MobileNet V2
base_model = tf.keras.applications.MobileNetV2(input_shape=(img_height, img_width, 3),
                                               include_top=False,
                                               weights='imagenet')

base_model.trainable = False

# Add classification head
num_classes = len(train_dataset.class_names)

model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(20)
])

model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ mobilenetv2_1.00_224            │ ?                      │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_1      │ ?                      │   0 (unbuilt) │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,257,984 (8.61 MB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 2,257,984 (8.61 MB)

In [13]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

epochs = 10
history = model.fit(train_dataset, validation_data=test_dataset, epochs=epochs)

Epoch 1/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 12s 366ms/step - accuracy: 0.0489 - loss: 3.2670 - val_accuracy: 0.1910 - val_loss: 2.8911
Epoch 2/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 8s 304ms/step - accuracy: 0.2748 - loss: 2.3977 - val_accuracy: 0.2462 - val_loss: 2.6723
Epoch 3/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 8s 307ms/step - accuracy: 0.3832 - loss: 2.0470 - val_accuracy: 0.2965 - val_loss: 2.6021
Epoch 4/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 8s 315ms/step - accuracy: 0.4901 - loss: 1.7978 - val_accuracy: 0.3467 - val_loss: 2.5006
Epoch 5/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 8s 313ms/step - accuracy: 0.5550 - loss: 1.5831 - val_accuracy: 0.3769 - val_loss: 2.4597
Epoch 6/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 8s 308ms/step - accuracy: 0.5770 - loss: 1.4654 - val_accuracy: 0.4372 - val_loss: 2.4536
Epoch 7/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 8s 319ms/step - accuracy: 0.6327 - loss: 1.3826 - val_accuracy: 0.4121 - val_loss: 2.4434
Epoch 8/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 8s 310ms/step - accuracy: 0.6676 - loss: 1.2783 - val_accuracy: 0

In [14]:
loss, accuracy = model.evaluate(test_dataset)
print(f'Test accuracy: {accuracy}')

7/7 ━━━━━━━━━━━━━━━━━━━━ 2s 233ms/step - accuracy: 0.4895 - loss: 2.4014
Test accuracy: 0.45728641748428345


: 

### Convert the Model to TFLite:



In [15]:
# Convert the model
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the model
with open('model_david.tflite', 'wb') as f:
    f.write(tflite_model)

print("Model converted to TFLite and saved as 'model.tflite'")

INFO:tensorflow:Assets written to: C:\Users\david\AppData\Local\Temp\tmpcnonycos\assets


INFO:tensorflow:Assets written to: C:\Users\david\AppData\Local\Temp\tmpcnonycos\assets
